In [1]:
from utils import load_config
from llm import *
from prompt import *
from harness import *
import os
original_directory = '/home/sagemaker-user/harnessgeneration'

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
import subprocess
import time
from datetime import datetime
def has_coverage(config):
    seconds_to_wait = 5
    print(f'checking that there is some coverage (Running the fuzzer for {seconds_to_wait}) seconds')
    # print(datetime.now())
    # r = subprocess.Popen(['afl-fuzz', '-i', 'seed', '-o', 'out', '--', f'./{HARNESS_BIN}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # time.sleep(seconds_to_wait)
    # r.terminate()
    # try:
    #     r.wait(timeout=2)  # Give some time to clean up resources
    # except subprocess.TimeoutExpired:
    #     r.kill()  # Force kill if it's still not terminating
    # print(datetime.now())
    print(datetime.now())
    
    cmd = f"timeout {seconds_to_wait}s afl-fuzz -i {config['seeds']} -o out -- ./{config['harness_bin']} > /dev/null"
    get_ipython().system(cmd)
    # !timeout 5s afl-fuzz -i config['seeds'] -o out -- config['harness_bin'] #> /dev/null
    # time.sleep(seconds_to_wait)
    print(datetime.now())
    
    result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stats = extract_summary_stats(result.stdout)
    print(f"{stats['coverage reached'][:-1]}")
    if float(stats['coverage reached'][:-1]) == 0:
        print('Trying to fuzz for 60 seconds')
        seconds_to_wait = 60
        cmd = f"timeout {seconds_to_wait}s afl-fuzz -i {config['seeds']} -o out -- ./{config['harness_bin']} > /dev/null"
        get_ipython().system(cmd)
        # !timeout 60s afl-fuzz -i seed -o out -- ./harness > /dev/null
        result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stats = extract_summary_stats(result.stdout)
        print(f"{stats['coverage reached'][:-1]}=")
    return not float(stats['coverage reached'][:-1]) == 0

In [4]:
# GENERATE HARNESS
def generate_harness(new_dir, compile_command, CONFIG):
    
    os.chdir(original_directory)
    os.chdir(original_directory + new_dir)
    
    tentatives = 10
    initial_prompt = generate_initial_prompt(CONFIG)
    first_answer = get_answer(initial_prompt)
    extract_and_save_harness(first_answer, CONFIG['harness'])
    prev_prompt = initial_prompt
    prev_res = first_answer

    while tentatives:
        print(f"{tentatives=}")
        tentatives = tentatives -1
        # checking that harness is compilable
        compile_res = compile(compile_command)
        if compile_res[0]:
        # checking that does not crash on seed
            run_res = run_seed(CONFIG)
            if run_res[0]:
                # checking that there is some coverage
                if has_coverage(CONFIG):
                    break
                else:
                    prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "When running afl-fuzz over the generated code, the Coverage reached value is 0%. It probably means that the input from the fuzzer is not correclty passed to the fuzzed function. Can you regenerate the code to fiX this issue?\n")
                    prev_res = get_answer(prev_prompt)
                    extract_and_save_harness(prev_res, CONFIG['harness'])
            else:
                print(f"{run_res[1]=}")
                prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "The generated code when run with input AAAA is crashing with this error\n" + run_res[1] + '\ncan you regenerate the previous code to fix the issue?')
                prev_res = get_answer(prev_prompt)
                extract_and_save_harness(prev_res, CONFIG['harness'])
        else:
            print(f"{compile_res[1]=}")
            prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "can you regenerate the previous code to fix the following compilation error:\n" + compile_res[1])
            prev_res = get_answer(prev_prompt)
            extract_and_save_harness(prev_res, CONFIG['harness'])
            # break
    print("New harness saved in" + original_directory + '/evaluation/simple_lib/src/' + CONFIG['harness'])
    os.chdir(original_directory)

In [9]:
# GENERATE HARNESS
def generate_harness_by_example(new_dir, compile_command, CONFIG):
    
    os.chdir(original_directory)
    os.chdir(original_directory + new_dir)
    
    tentatives = 10
    initial_prompt = generate_initial_prompt_by_example(CONFIG)
    first_answer = get_answer(initial_prompt)
    extract_and_save_harness(first_answer, CONFIG['harness'])
    prev_prompt = initial_prompt
    prev_res = first_answer

    while tentatives:
        print(f"{tentatives=}")
        tentatives = tentatives -1
        # checking that harness is compilable
        compile_res = compile(compile_command)
        if compile_res[0]:
        # checking that does not crash on seed
            run_res = run_seed(CONFIG)
            if run_res[0]:
                # checking that there is some coverage
                if has_coverage(CONFIG):
                    break
                else:
                    prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "When running afl-fuzz over the generated code, the Coverage reached value is 0%. It probably means that the input from the fuzzer is not correclty passed to the fuzzed function. Can you regenerate the code to fiX this issue?\n")
                    prev_res = get_answer(prev_prompt)
                    extract_and_save_harness(prev_res, CONFIG['harness'])
            else:
                print(f"{run_res[1]=}")
                prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "The generated code when run with input AAAA is crashing with this error\n" + run_res[1] + '\ncan you regenerate the previous code to fix the issue?')
                prev_res = get_answer(prev_prompt)
                extract_and_save_harness(prev_res, CONFIG['harness'])
        else:
            print(f"{compile_res[1]=}")
            prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "can you regenerate the previous code to fix the following compilation error:\n" + compile_res[1])
            prev_res = get_answer(prev_prompt)
            extract_and_save_harness(prev_res, CONFIG['harness'])
            # break
    print(f"New harness saved in" + original_directory + '/evaluation/simple_lib/src/' + CONFIG['harness'])
    os.chdir(original_directory)

# Simple_lib

## lib_mul

In [4]:
config_file= "/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/lib_mul.yaml"
CONFIG = load_config(config_file)
new_dir = '/evaluation/simple_lib/src/'
compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', CONFIG['harness'], 'library.c', '-o', CONFIG['harness_bin']]
generate_harness(new_dir, compile_command, CONFIG)

  ```
    #include "library.h"
    #include <stdint.h>
    
    int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
      int x = *(int*)Data;
      int y = *(int*)(Data + sizeof(int));
      lib_mul(x, y);
      return 0;
    }
    ```
tentatives=10
checking that harness is compilable
compile_command=['afl-gcc-fast', '-fsanitize=fuzzer', 'harness_lib_mul.c', 'library.c', '-o', 'harness_lib_mul']
Compilation successful.
checking that does not crash on seed
checking that there is some coverage (Running the fuzzer for 5) seconds
2024-04-20 06:28:18.891008
2024-04-20 06:28:24.073056
0.01
New harness saved in/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/src/harness_lib_mul.c


## lib_echo

In [5]:
config_file= "/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/lib_echo.yaml"
CONFIG = load_config(config_file)
new_dir = '/evaluation/simple_lib/src/'
compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', CONFIG['harness'], 'library.c', '-o', CONFIG['harness_bin']]
generate_harness(new_dir, compile_command, CONFIG)

  ```c
    #include "library.h"
    #include <stdint.h>
    #include <stdlib.h>
    
    int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
      lib_echo((char *)Data, Size);
      return 0;
    }
    ```
C code has been extracted and saved to harness_lib_echo.c
tentatives=10
checking that harness is compilable
compile_command=['afl-gcc-fast', '-fsanitize=fuzzer', 'harness_lib_echo.c', 'library.c', '-o', 'harness_lib_echo']
Compilation successful.
checking that does not crash on seed
checking that there is some coverage (Running the fuzzer for 5) seconds
2024-04-22 07:41:37.711914
2024-04-22 07:41:42.873142
0.02
New harness saved in/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/src/harness_lib_echo.c


# Libxml2

## xmlReadFile

In [10]:
config_file= "/home/sagemaker-user/harnessgeneration/evaluation/new-libxml2/xmlReadFile.yaml"
CONFIG = load_config(config_file)
new_dir = '/evaluation/new-libxml2/'
compile_command = CONFIG['compile_command'].split()
generate_harness_by_example(new_dir, compile_command, CONFIG)

  Here is a possible implementation of the libFuzzer target for the `xmlReadFile` function:
```c
#include <stdint.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

#include <libxml/xmlreader.h>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    xmlTextReaderPtr reader = xmlReaderForMemory(Data, Size, NULL, NULL, 0);
    if (reader == NULL) {
        return 0;
    }

    while (xmlTextReaderRead(reader) == 1) {
        const xmlChar *name = xmlTextReaderConstName(reader);
        if (name == NULL) {
            continue;
        }

        // Step 3: insert code to handle the element name
        <FILL>;
    }

    xmlFreeTextReader(reader);
    return 0;
}
```
This implementation uses the `xmlReaderForMemory` function to create an `xmlTextReaderPtr` from the input data, and then uses the `xmlTextReaderRead` function to iterate over the elements in the document. For each element, it retrieves the element name using the `xmlTextReaderConstName` function, and

## xmlDocGetRootElement

In [12]:
config_file= "/home/sagemaker-user/harnessgeneration/evaluation/new-libxml2/xmlDocGetRootElement.yaml"
CONFIG = load_config(config_file)
new_dir = '/evaluation/new-libxml2/'
compile_command = CONFIG['compile_command'].split()
generate_harness_by_example(new_dir, compile_command, CONFIG)

  [C]
#include <libxml/parser.h>
#include <libxml/xmlmemory.h>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    xmlDocPtr doc = xmlReadMemory((const char *)Data, Size, "", NULL, 0);
    if (doc == NULL) {
        return 0;
    }
    xmlNodePtr root = xmlDocGetRootElement(doc);
    if (root == NULL) {
        xmlFreeDoc(doc);
        return 0;
    }
    xmlFreeDoc(doc);
    return 0;
}
[/C]

C code has been extracted and saved to harness_xmlDocGetRootElement.c
tentatives=10
checking that harness is compilable
compile_command=['afl-gcc-fast', '-fsanitize=fuzzer', './harness_xmlDocGetRootElement.c', '-I', 'libxml2/include', 'libxml2/.libs/libxml2.a', '-lz', '-lm', '-o', 'harness_xmlDocGetRootElement']
Compilation failed.
Errors: ./harness_xmlDocGetRootElement.c:4:34: error: unknown type name ‘uint8_t’
    4 | int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
      |                                  ^~~~~~~

compile_res[1]='./harness_xmlDocGetRootElement.c:4

# libarchive

In [13]:
config_file= "/home/sagemaker-user/harnessgeneration/evaluation/libarchive/archive_read_data.yaml"
CONFIG = load_config(config_file)
new_dir = '/evaluation/libarchive/'
compile_command = CONFIG['compile_command'].split()
generate_harness_by_example(new_dir, compile_command, CONFIG)

  [C]
#include <archive.h>
#include <archive_entry.h>
#include <stdint.h>
#include <stdlib.h>
#include <stdio.h>
#include <string.h>

int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
  struct archive *a = archive_read_new();
  struct archive_entry *entry;
  int r = archive_read_support_filter_all(a);
  r = archive_read_support_format_all(a);
  r = archive_read_open_memory(a, Data, Size);
  while ((r = archive_read_next_header(a, &entry)) == ARCHIVE_OK) {
    archive_read_data(a, Data, Size);
  }
  archive_read_free(a);
  return 0;
}
[/C]
This is a C/C++ libFuzzer target for the archive_read_data function of the c library. It includes the necessary headers, defines the LLVMFuzzerTestOneInput function, and calls the archive_read_data function with the input data.

The first step is to include the necessary headers. In this case, we need to include the archive.h and archive_entry.h headers, as well as the standard headers stdint.h, stdlib.h, and string.h.

The second step is